In [ ]:
import os
import time
import imgaug as ia
import tensorflow as tf
from tensorflow import keras

## RandAugment Is Suck in TensorFlow
### Create TensorFlow `Dataset` objects
- https://keras.io/examples/vision/randaugment/#create-tensorflow-dataset-objects

- Because `RandAugment` can only process NumPy arrays, it cannot be applied directly as part of the `Dataset` object (which expects TensorFlow tensors). To make `RandAugment` part of the dataset, we need to wrap it in a [tf.py_function](https://www.tensorflow.org/api_docs/python/tf/py_function).

- A `tf.py_function` is a TensorFlow operation (which, like any other TensorFlow operation, takes TF tensors as arguments and returns TensorFlow tensors) that is capable of running arbitrary Python code. Naturally, this Python code can only be executed on CPU (whereas the rest of the TensorFlow graph can be accelerated on GPU), which in some cases can cause significant slowdowns -- however, in this case, the `Dataset` pipeline will run asynchronously together with the model, and doing preprocessing on CPU will remain performant.

In [ ]:
# for device
device_index = -1
MIXED_PRECISION_FLAG = True
## TF 2.6 does not support jit_compile in keras.Model.compile() yet.
## So, just set it to False.
## Another way is to use environment variable 'TF_XLA_FLAGS'.
## Set os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit'.
JIT_COMPILE_FLAG = True

# for dataset
dataset = 'imagenet'
dir_path = '/ssd'
## r_b = [(160, 510), (224, 390), (288, 180)] without jit_compile
## r_b = [(160, 510), (224, 360), (288, 170)] with jit_compile
resolution = 224
batch_size = 512

# for model
depth = 18
dropout_rate = 0.2 # [0.1, 0.2, 0.3]

# for training
learning_rate = 1e-1
momentum = 0.9
epochs = 120 # 120
## TF 2.6 does not support weight_decay in keras.optimizers.SGD() yet.
## So, it might be set in the model.
weight_decay = 1e-4

# for learning rate scheduler
milestones = [30, 60, 90] # [30, 60, 90]
gamma = 0.1

######## for testing: BS and LR are propotional
learning_rate *= batch_size / 128
######## for testing: kernel_regularizer for TF 2.6
KR_FLAG = True
KR_VALUE = keras.regularizers.L2(weight_decay) if KR_FLAG else None
######## for testing: RandAugment
JIT_COMPILE_FLAG = False # jit_compile "must" be "False"
num_augment = 1
magnitude_augment = 10 # [5, 10 ,15]

In [ ]:
######## for testing: TF 2.6 jit_compile, it must call berfore any tensorflow function.
## For unknown reasons, '--tf_xla_cpu_global_jit' only supports the first GPU.
## Otherwise an error will result.
if JIT_COMPILE_FLAG:
    if device_index == 0:
        # can not use the condition 'len(tf.config.list_physical_devices('GPU')) == 1'
        # since it calls tf function...
        os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit'
    else:
        os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2'

if MIXED_PRECISION_FLAG:
    policy = keras.mixed_precision.Policy('mixed_float16')
    keras.mixed_precision.set_global_policy(policy)
    print(f'Policy: {policy.name}')
    print(f'Compute dtype: {policy.compute_dtype}')
    print(f'Variable dtype: {policy.variable_dtype}')

print('----')
print(f'MIXED_PRECISION: {MIXED_PRECISION_FLAG}')
print(f'JIT_COMPILE: {JIT_COMPILE_FLAG}')
print('----')

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print(f'Numbers of Physical Devices: {len(physical_devices)}')
tf.config.set_visible_devices(physical_devices[device_index], 'GPU')
tf.config.experimental.set_memory_growth(physical_devices[device_index], True)
print(f'Using Device: {physical_devices[device_index]}')

In [ ]:
def load_imagenet(
    resolution: int, batch_size: int,
    num_augment: int = 2, magnitude_augment: int = 10
):
    mean = [0.485, 0.456, 0.406]
    std = [0.299, 0.224, 0.225]
    var = [0.089401, 0.050176, 0.050625] # tf.math.square(std)
    
    resolution_list = [160, 224, 288]
    
    if resolution not in resolution_list:
        raise ValueError(f'Invalid resolution "{resolution}", it should be in {resolution_list}.')
    
    '''
    # tf.keras.utils.image_dataset_from_directory() can not allow simple augmentation pipeline
    # simple augmentation pipeline == keras.layers.RandomXXX()
    # move simple augmentation pipeline to build_model
    simple_aug = keras.Sequential([
        keras.layers.RandomFlip('horizontal'),
        keras.layers.RandomRotation(factor=0.02),
        keras.layers.RandomZoom(height_factor=0.2, width_factor=0.2)
    ])
    '''
    def preprocessing_map(image):
        transform = keras.Sequential([
            keras.layers.Rescaling(1/255),
            keras.layers.Normalization(mean=mean, variance=var)
        ])
        return transform(image)
    
    rand_aug = ia.augmenters.RandAugment(n=num_augment, m=magnitude_augment)
    def augment(images):
        # Input to `augment()` is a TensorFlow tensor which is not supported by `imgaug`.
        # This is why we first convert it to its `numpy` variant.
        images = tf.cast(images, tf.uint8)
        return rand_aug(images=images.numpy())
    
    dataloader = {
        'train': (
            tf.keras.utils.image_dataset_from_directory(
                directory=f'{dir_path}/imagenet/train',
                label_mode='int', # for keras.losses.SparseCategoricalCrossentropy()
                batch_size=batch_size,
                image_size=(resolution, resolution)
            )
            .map(
                lambda x, y: (preprocessing_map(x), y),
                num_parallel_calls=tf.data.AUTOTUNE
            )
            # tf.data.cache() is a bomb, causing excessive memory usage when training imagenet
            .map(
                lambda x, y: (tf.py_function(augment, [x], [tf.float32])[0], y),
                num_parallel_calls=tf.data.AUTOTUNE
            )
            .prefetch(buffer_size=tf.data.AUTOTUNE)
        ),
        'val': (
            tf.keras.utils.image_dataset_from_directory(
                directory=f'{dir_path}/imagenet/val',
                label_mode='int', # for keras.losses.SparseCategoricalCrossentropy()
                batch_size=batch_size,
                image_size=(resolution, resolution)
            )
            .map(
                lambda x, y: (preprocessing_map(x), y),
                num_parallel_calls=tf.data.AUTOTUNE
            )
            # tf.data.cache() is a bomb, causing excessive memory usage when training imagenet
            .prefetch(buffer_size=tf.data.AUTOTUNE)
        )
    }
    
    return dataloader

In [ ]:
def build_resnet(
    dataset: str,
    depth: int,
    dropout_rate: float,
    resolution: int
) -> keras.Model:
    
    dataset_list = ['cifar10', 'cifar100', 'imagenet']
    depth_list = [18, 34]
    
    if dataset not in dataset_list:
        raise ValueError(f'Invalid dataset "{dataset}", it should be in {dataset_list}.')
    if depth not in depth_list:
        raise ValueError(f'Invalid depth "{depth}", it should be in {depth_list}.')
    
    if dataset == 'cifar10':
        classes = 10
    elif dataset == 'cifar100':
        classes = 100
    elif dataset == 'imagenet':
        classes = 1000
    
    if depth == 18:
        stack_list = [2, 2, 2, 2]
    elif depth == 34:
        stack_list = [3, 4, 6, 3]
    
    def basic_block(x: keras.Input, filters: int, conv_shortcut: bool = False):
        if conv_shortcut:
            shortcut = keras.layers.Conv2D(
                filters, 1, strides=2, use_bias=False,
                kernel_initializer='he_normal',
                kernel_regularizer=KR_VALUE
            )(x)
            shortcut = keras.layers.BatchNormalization(momentum=0.9, epsilon=1.001e-5)(shortcut)
            x = keras.layers.Conv2D(
                filters, 3, strides=2, padding='same', use_bias=False,
                kernel_initializer='he_normal',
                kernel_regularizer=KR_VALUE
            )(x)
        else:
            shortcut = x
            x = keras.layers.Conv2D(
                filters, 3, padding='same', use_bias=False,
                kernel_initializer='he_normal',
                kernel_regularizer=KR_VALUE
            )(x)
        x = keras.layers.BatchNormalization(momentum=0.9, epsilon=1.001e-5)(x)
        x = keras.layers.Activation('relu')(x)
        x = keras.layers.Conv2D(
            filters, 3, padding='same', use_bias=False,
            kernel_initializer='he_normal',
            kernel_regularizer=KR_VALUE
        )(x)
        x = keras.layers.BatchNormalization(momentum=0.9, epsilon=1.001e-5)(x)
        x = keras.layers.Add()([shortcut, x])
        x = keras.layers.Activation('relu')(x)
        return x
    
    def basic_stack(x: keras.Input, filters: int, stack: int, conv_shortcut: bool = False):
        for i in range(stack):
            if i == 0 and conv_shortcut == True:
                filters *= 2
                x = basic_block(x, filters, conv_shortcut)
            else:
                x = basic_block(x, filters)
        return x, filters
    
    inputs = keras.Input(shape=(resolution, resolution, 3))
    filters = 64
    '''
    ## simple augmentation pipeline
    simple_aug = keras.Sequential([
        keras.layers.RandomFlip('horizontal'),
        keras.layers.RandomRotation(factor=0.02),
        keras.layers.RandomZoom(height_factor=0.2, width_factor=0.2)
    ])
    x = simple_aug(inputs)
    '''
    ## random augmentation pipeline is included in data preprocessing
    x = inputs
    ## stem
    if 'cifar' in dataset:
        x = keras.layers.Conv2D(
            filters, 3, padding='same', use_bias=False,
            kernel_initializer='he_normal',
            kernel_regularizer=KR_VALUE
        )(x)
        x = keras.layers.BatchNormalization(momentum=0.9, epsilon=1.001e-5)(x)
        x = keras.layers.Activation('relu')(x)
    elif dataset == 'imagenet':
        x = keras.layers.Conv2D(
            filters, 7, strides=2, padding='same', use_bias=False,
            kernel_initializer='he_normal',
            kernel_regularizer=KR_VALUE
        )(x)
        x = keras.layers.BatchNormalization(momentum=0.9, epsilon=1.001e-5)(x)
        x = keras.layers.Activation('relu')(x)
        x = keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    ## trunk
    for i, stack in enumerate(stack_list):
        if i == 0:
            x, filters = basic_stack(x, filters, stack)
        else:
            x, filters = basic_stack(x, filters, stack, True)
    ## classifier
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(dropout_rate)(x)
    outputs = keras.layers.Dense(
        classes, activation='softmax',
        kernel_regularizer=KR_VALUE
    )(x)
    
    return keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
dataloader = load_imagenet(
    resolution=resolution, batch_size=batch_size,
    num_augment=num_augment, magnitude_augment=magnitude_augment
)

In [ ]:
model = build_resnet(dataset=dataset, depth=depth, dropout_rate=dropout_rate, resolution=resolution)
#print(model.summary())

In [ ]:
def lr_schedule(epoch, lr, milestones, gamma: float = 0.1):
    if epoch in milestones:
        lr *= gamma
    return lr

class TimeCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.history = []
    def on_epoch_begin(self, epoch, logs=None):
        self.time_epoch_begin = time.perf_counter()
    def on_epoch_end(self, epoch, logs=None):
        self.history.append(time.perf_counter() - self.time_epoch_begin)

lr_scheduler_callback = keras.callbacks.LearningRateScheduler(
    lambda x, y: lr_schedule(x, y, milestones=milestones, gamma=gamma)
)

time_callback = TimeCallback()

In [ ]:
model.compile(
    optimizer=keras.optimizers.SGD(
        learning_rate=learning_rate,
        momentum=momentum,
        #weight_decay=weight_decay
        # `decay_steps` in `keras.optimizers.schedules.LearningRateSchedule`
        # means batches instead of epochs, which is a fine grained value,
        # so try to use `keras.callbacks.LearningRateScheduler`
        # to set the learning rate decay value each epoch.
    ),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    #jit_compile=JIT_COMPILE_FLAG
)

In [ ]:
logs = model.fit(
    dataloader['train'],
    epochs=epochs,
    verbose='auto',
    callbacks=[time_callback, lr_scheduler_callback],
    validation_data=dataloader['val']
)
logs.history['t'] = time_callback.history